# Nifti Read Example

The purpose of this notebook is to illustrate reading Nifti files and iterating over patches of the volumes loaded from them.

In [1]:
%matplotlib inline

import os
import sys
from glob import glob
import tempfile

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib


import torch
from torch.utils.data import DataLoader
from monai.transforms.compose import Compose

import monai

from monai.data.nifti_reader import NiftiDataset
from monai.transforms import AddChannel, Transpose, Rescale, ToTensor, RandUniformPatch
from monai.data.grid_dataset import GridPatchDataset
from monai.data.synthetic import create_test_image_3d

monai.config.print_config()

MONAI version: 0.0.1
Python version: 3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 16:30:03)  [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Numpy version: 1.18.2
Pytorch version: 1.4.0
Ignite version: 0.3.0


Create a number of test Nifti files:

In [2]:
tempdir = tempfile.mkdtemp()

for i in range(5):
    im, seg = create_test_image_3d(128, 128, 128)
    
    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'im%i.nii.gz'%i))
    
    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'seg%i.nii.gz'%i))

Create a data loader which yields uniform random patches from loaded Nifti files:

In [3]:
images = sorted(glob(os.path.join(tempdir,'im*.nii.gz')))
segs = sorted(glob(os.path.join(tempdir,'seg*.nii.gz')))

imtrans = Compose([
    Rescale(),
    AddChannel(),
    RandUniformPatch((64, 64, 64)),
    ToTensor()
])    

segtrans = Compose([
    AddChannel(),
    RandUniformPatch((64, 64, 64)),
    ToTensor()
])    
    
ds = NiftiDataset(images, segs, transform=imtrans, seg_transform=segtrans)

loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = monai.utils.misc.first(loader)
print(im.shape, seg.shape)

torch.Size([5, 1, 64, 64, 64]) torch.Size([5, 1, 64, 64, 64])


Alternatively create a data loader which yields patches in regular grid order from loaded images:

In [4]:
imtrans = Compose([
    Rescale(),
    AddChannel(),
    ToTensor()
])    

segtrans = Compose([
    AddChannel(),
    ToTensor()
])    
    
ds = NiftiDataset(images, segs, transform=imtrans, seg_transform=segtrans)
ds = GridPatchDataset(ds, (64, 64, 64))

loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = monai.utils.misc.first(loader)
print(im.shape, seg.shape)

torch.Size([10, 1, 64, 64, 64]) torch.Size([10, 1, 64, 64, 64])


In [5]:
!rm -rf {tempdir}